In [38]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [39]:
!pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [40]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [41]:
from transformers import AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
import torch
import numpy as np
import evaluate  # pip install evaluate
import seqeval   # pip install seqeval
from datasets import load_dataset

In [42]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')
print(tokenizer)

BertTokenizerFast(name_or_path='google-bert/bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)


In [43]:
dataset_name = "doushabao4766/msra_ner_k_V3"
my_dataset = load_dataset(dataset_name)
print(my_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 3443
    })
})


In [44]:
train_dataset = my_dataset['train']
print(train_dataset)
print()

print(train_dataset.features)
print()

print(train_dataset[0])

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'knowledge'],
    num_rows: 45001
})

{'id': Value(dtype='string', id=None), 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None), 'knowledge': Value(dtype='string', id=None)}

{'id': '0', 'tokens': ['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'knowledge': ''}


In [45]:
# 原始文本转换模型需要token_idx,生成和 token_idx对齐label
def data_input_proc(items):
    text = [''.join(token) for token in items['tokens']]
    input_data = tokenizer(
        text, 
        truncation=True, 
        add_special_tokens=False,
        max_length=512,
    )

    adjust_labels = []  # 所有修正后label索引列表
    # 上一步骤生成ner_tag中索引和token对齐
    for k in range(len(input_data['input_ids'])):
        # 每条记录token对应word_ids
        word_ids = input_data.word_ids(k)
        # 批次ner_tag长度和token长度对齐
        tags = items['ner_tags'][k]

        adjusted_label_ids = []
        i, prev_word_id = -1,-1
        for word_id in word_ids:
            if (word_id != prev_word_id):   #  word_ids [0, 1, 2, 3, 4, 4]
                i += 1 # token对应检索位置+1
                prev_word_id = word_id
            if i < len(tags):
                adjusted_label_ids.append(tags[i])
            else:
                # 处理超出范围的情况，例如填充默认标签
                adjusted_label_ids.append(-100)  # 假设 -100 是填充标签
        adjust_labels.append(adjusted_label_ids)
    # 修正后label添加到input_data
    input_data['labels'] = adjust_labels
    return input_data

my_dataset1 = my_dataset.map(data_input_proc, batched=True)  # batched 每次传入自定义方法样本数量多个
print(my_dataset1)

Map:   0%|          | 0/3443 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3443
    })
})


In [46]:
print(my_dataset1["train"][0]["labels"])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [47]:
my_dataset1.set_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

print(my_dataset1["train"][0])

{'input_ids': tensor([2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636,  674, 1036, 4997,
        2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768,
        7599, 3198, 8024,  791, 1921, 3300, 3119, 5966,  817,  966, 4638,  741,
         872, 3766,  743, 8024, 3209, 3189, 2218, 1373,  872, 2637,  679, 2496,
        1159, 8013]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1]), 'labels': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0])}


In [48]:
ClassLabels = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

id2label = {i:ClassLabel for i, ClassLabel in enumerate(ClassLabels)}
label2id = {ClassLabel:i for i, ClassLabel in enumerate(ClassLabels)}

model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese',
                                                        num_labels=len(ClassLabels),
                                                        id2label=id2label,
                                                        label2id=label2id)
print(model)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [51]:
args = TrainingArguments(
    output_dir="ner_train",  # 模型训练工作目录（tensorboard，临时模型存盘文件，日志）
    num_train_epochs = 6,    # 训练 epoch
    save_safetensors=False,  # 设置 False 保存文件可以通过torch.load加载
    per_device_train_batch_size=32,  # 训练批次
    per_device_eval_batch_size=32,   # 验证批次
    report_to='tensorboard',  # 训练输出记录
    eval_strategy="epoch",
)

# metric 方法
def compute_metric(result):
    # result 是一个tuple (predicts, labels)

    # 获取评估对象
    seqeval = evaluate.load('seqeval')
    predicts,labels = result
    predicts = np.argmax(predicts, axis=2)

    # 准备评估数据
    predicts = [[ClassLabels[p] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    labels = [[ClassLabels[l] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    results = seqeval.compute(predictions=predicts, references=labels)

    return results

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True)

trainer = Trainer(
    model,
    args,
    train_dataset=my_dataset1['train'],
    eval_dataset=my_dataset1['test'],
    data_collator=data_collator,
    compute_metrics=compute_metric
)

In [52]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Loc,Org,Per,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.101400,0.049429,"{'precision': 0.8584392014519057, 'recall': 0.8306989813839128, 'f1': 0.8443413066761871, 'number': 2847}","{'precision': 0.7329149232914923, 'recall': 0.7968157695223654, 'f1': 0.7635306937885943, 'number': 1319}","{'precision': 0.8587859424920128, 'recall': 0.8918380889183809, 'f1': 0.8750000000000001, 'number': 1507}",0.827251,0.839062,0.833115,0.983566
2,0.046200,0.051089,"{'precision': 0.8422770123277737, 'recall': 0.8159466104671584, 'f1': 0.8289027653880465, 'number': 2847}","{'precision': 0.8058035714285714, 'recall': 0.821076573161486, 'f1': 0.8133683815245963, 'number': 1319}","{'precision': 0.8677524429967427, 'recall': 0.8838752488387525, 'f1': 0.8757396449704141, 'number': 1507}",0.840518,0.835184,0.837843,0.984703
3,0.024700,0.045283,"{'precision': 0.8887692859705777, 'recall': 0.8700386371619249, 'f1': 0.8793042243521477, 'number': 2847}","{'precision': 0.8295964125560538, 'recall': 0.8415466262319939, 'f1': 0.8355287918705306, 'number': 1319}","{'precision': 0.8981058131939909, 'recall': 0.9124087591240876, 'f1': 0.9052007899934167, 'number': 1507}",0.877298,0.874669,0.875982,0.988060
4,0.018300,0.048601,"{'precision': 0.8705592683784734, 'recall': 0.8693361433087461, 'f1': 0.8699472759226713, 'number': 2847}","{'precision': 0.8107511045655376, 'recall': 0.8347232752084913, 'f1': 0.8225625700410909, 'number': 1319}","{'precision': 0.8835526315789474, 'recall': 0.8911745189117452, 'f1': 0.8873472084572184, 'number': 1507}",0.859815,0.867090,0.863437,0.987483
5,0.012800,0.048968,"{'precision': 0.9179856115107914, 'recall': 0.8963821566561293, 'f1': 0.9070552692376045, 'number': 2847}","{'precision': 0.8412816691505216, 'recall': 0.8559514783927218, 'f1': 0.8485531754979331, 'number': 1319}","{'precision': 0.917989417989418, 'recall': 0.9210351692103517, 'f1': 0.9195097714474992, 'number': 1507}",0.899716,0.893531,0.896613,0.989215
6,0.010700,0.049934,"{'precision': 0.913867047891351, 'recall': 0.8981383912890762, 'f1': 0.9059344552701506, 'number': 2847}","{'precision': 0.8404726735598228, 'recall': 0.8627748294162244, 'f1': 0.8514777403666294, 'number': 1319}","{'precision': 0.9215039577836411, 'recall': 0.927007299270073, 'f1': 0.9242474363215349, 'number': 1507}",0.898377,0.897585,0.897981,0.989468


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Trainer is attempting to log a value of "{'precision': 0.8584392014519057, 'recall': 0.8306989813839128, 'f1': 0.8443413066761871, 'number': 2847}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7329149232914923, 'recall': 0.7968157695223654, 'f1': 0.7635306937885943, 'number': 1319}" of type <class 'dict'> for key "eval/ORG" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8587859424920128, 'recall': 0.8918380889183809, 'f1': 0.8750000000000001, 'number': 1507}" of type <class 'dict'> for ke

TrainOutput(global_step=4224, training_loss=0.031716337032390365, metrics={'train_runtime': 4843.5188, 'train_samples_per_second': 55.746, 'train_steps_per_second': 0.872, 'total_flos': 2.349446673461672e+16, 'train_loss': 0.031716337032390365, 'epoch': 6.0})

In [54]:
def predict_entities(text):
    # 预处理输入文本
    input_data = tokenizer(
        text, 
        return_tensors="pt",
        truncation=True, 
        add_special_tokens=False,
        max_length=512,
    )

    # 确保数据和模型在同一设备上
    device = next(model.parameters()).device  # 获取模型所在的设备
    for key in input_data:
        if isinstance(input_data[key], torch.Tensor):
            input_data[key] = input_data[key].to(device)

    # 运行模型推理
    with torch.no_grad():
        outputs = model(**input_data)

    # 获取预测结果
    predictions = torch.argmax(outputs.logits, dim=2).squeeze().tolist()

    # 获取分词后的 token
    tokens = tokenizer.convert_ids_to_tokens(input_data["input_ids"].squeeze().tolist())

    # 后处理输出，提取实体
    entities = []
    current_entity = None
    current_label = None

    for i, (token, pred) in enumerate(zip(tokens, predictions)):
        label = id2label[pred]

        if label.startswith("B-"):
            if current_entity is not None:
                entities.append(current_entity)
            current_entity = {"entity": label[2:], "content": token}
            current_label = label
        elif label.startswith("I-") and current_entity is not None and label[2:] == current_entity["entity"]:
            current_entity["content"] += token
        else:
            if current_entity is not None:
                entities.append(current_entity)
                current_entity = None

    # 添加最后一个实体（如果有）
    if current_entity is not None:
        entities.append(current_entity)

    return entities

# 示例调用
text = "双方确定了今后发展中美关系的指导方针。"
entities = predict_entities(text)
print(entities)

[{'entity': 'LOC', 'content': '中'}, {'entity': 'LOC', 'content': '美'}]
